In [9]:
print("hi)")


hi)


In [6]:
import numpy as np
import pickle, gzip
import pandas as pd
import matplotlib.pyplot as plt
import torch
import time
import os
import urllib.request

In [7]:
data = pd.read_csv("smoking_health_data_final.csv")

In [8]:
data.head()

age   sex current_smoker  heart_rate blood_pressure  cigs_per_day   chol
0   54  male            yes          95         110/72           NaN  219.0
1   45  male            yes          64         121/72           NaN  248.0
2   58  male            yes          81       127.5/76           NaN  235.0
3   42  male            yes          90       122.5/80           NaN  225.0
4   42  male            yes          62         119/80           NaN  226.0

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
# Handle missing values
# Impute missing values for numerical columns with the median
numerical_features = ['age', 'cigs_per_day', 'chol']
numerical_transformer = SimpleImputer(strategy='median')

# Convert categorical features
categorical_features = ['sex', 'current_smoker']
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])


# Define preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Define features and target
X = data.drop(columns=['blood_pressure'])
y = data['blood_pressure']

# Preprocess the data
X = preprocessor.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Output the shapes of the resulting datasets
(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

((3120, 7), (780, 7), (3120,), (780,))

In [10]:
print("Training Features:\n", X_train)
print("Test Features:\n", X_test)
print("Training Labels:\n", y_train)
print("Test Labels:\n", y_test)

Training Features:
 [[ 55.   0. 199. ...   1.   1.   0.]
 [ 51.  15. 267. ...   1.   0.   1.]
 [ 44.   0. 243. ...   1.   1.   0.]
 ...
 [ 41.   0. 235. ...   0.   1.   0.]
 [ 48.  25. 304. ...   0.   0.   1.]
 [ 42.  20. 319. ...   1.   0.   1.]]
Test Features:
 [[ 40.   0. 193. ...   1.   1.   0.]
 [ 59.   0. 236. ...   1.   1.   0.]
 [ 64.   0. 232. ...   0.   1.   0.]
 ...
 [ 53.  15. 233. ...   1.   0.   1.]
 [ 47.   9. 201. ...   1.   0.   1.]
 [ 37.   0. 218. ...   1.   1.   0.]]
Training Labels:
 247       134/80
2591    129.5/80
1822      146/91
3483      124/81
2771      122/87
          ...   
1130      148/74
1294    130.5/86
860       158/93
3507    102/66.5
3174      116/74
Name: blood_pressure, Length: 3120, dtype: object
Test Labels:
 839       130/89
1717      127/83
321     149.5/84
3187      128/74
2269    131.5/89
          ...   
1393      150/94
3751    103.5/60
2696      130/94
2384      122/67
805       130/89
Name: blood_pressure, Length: 780, dtype: object


In [11]:

train_set = pd.concat([pd.DataFrame(X_train), pd.Series(y_train, name='blood_pressure')], axis=1)

# Combine the features and labels for test set
test_set = pd.concat([pd.DataFrame(X_test), pd.Series(y_test, name='blood_pressure')], axis=1)


In [12]:
import os
import pandas as pd

# Check the current working directory
current_directory = os.getcwd()
#print("Current Directory:", current_directory)


In [13]:

train_set.to_csv('pproject_train.csv', index=False)

# Save the test set to CSV
test_set.to_csv('pproject_test.csv', index=False)

In [14]:
import pandas as pd
test_data = pd.read_csv("pproject_test.csv")

train_data = pd.read_csv("pproject_train.csv")

medians = train_data.median()
blood_pressure_mode = train_data['blood_pressure'].mode()[0]

for column in test_data.columns[:-1]:  # Adjust if 'blood_pressure' is not the last column
    test_data[column].fillna(medians[column], inplace=True)
    
test_data['blood_pressure'].fillna(blood_pressure_mode, inplace=True)

C:\Users\Mirka\AppData\Local\Temp\ipykernel_18580\4234110969.py:6: FutureWarning: The default value of numeric_only in DataFrame.median is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  medians = train_data.median()


In [15]:
test_data.info()
test_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1393 entries, 0 to 1392
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   0               1393 non-null   float64
 1   1               1393 non-null   float64
 2   2               1393 non-null   float64
 3   3               1393 non-null   float64
 4   4               1393 non-null   float64
 5   5               1393 non-null   float64
 6   6               1393 non-null   float64
 7   blood_pressure  1393 non-null   object 
dtypes: float64(7), object(1)
memory usage: 87.2+ KB


0     1      2    3    4    5    6 blood_pressure
0  40.0   0.0  193.0  0.0  1.0  1.0  0.0         110/72
1  59.0   0.0  236.0  0.0  1.0  1.0  0.0         130/80
2  64.0   0.0  232.0  1.0  0.0  1.0  0.0         130/80
3  48.0  20.0  202.0  1.0  0.0  0.0  1.0         130/80
4  51.0   6.0  190.0  1.0  0.0  0.0  1.0         130/80

In [16]:
test_data.to_csv('pproject_test.csv', index=False)

In [17]:
test_d = pd.read_csv("pproject_test.csv")
column_names = ['age', 'cigs_per_day', 'chol', 'male', 'female', 'smoker_yes', 'smoker_no', 'blood_pressure']
test_data.columns = column_names
print(test_data.head())
test_data.to_csv('pproject_test.csv', index=False)

    age  cigs_per_day   chol  male  female  smoker_yes  smoker_no  \
0  40.0           0.0  193.0   0.0     1.0         1.0        0.0   
1  59.0           0.0  236.0   0.0     1.0         1.0        0.0   
2  64.0           0.0  232.0   1.0     0.0         1.0        0.0   
3  48.0          20.0  202.0   1.0     0.0         0.0        1.0   
4  51.0           6.0  190.0   1.0     0.0         0.0        1.0   

  blood_pressure  
0         110/72  
1         130/80  
2         130/80  
3         130/80  
4         130/80  
